### math 510 project

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# constants

N = 200         # number of neurons (200)
M = 5           # number of external inputs to the network (?)
T = 10          # total time of the simulation (?)
delta_t = 0.01  # length of the time step (?)
tau = 0.03      # relaxation time constant (0.03)
g = 1.5         # scaling factor (1.5)

In [ ]:
# initialize the activations x to uniform noise in the range [-0.1, 0.1]

x = 0.1 * (2 * np.random.rand(N,1) - 1)

In [ ]:
# initialize the external inputs

u = np.zeros((M,1))

In [ ]:
# initialize the weights J, taken from normal distribution with
# mean 0 and standard deviation g / sqrt(N)

J = np.random.normal(loc=0.0, scale=(g/np.sqrt(N)), size=(N,N))

In [ ]:
# initialize the (external) weights B, taken from uniform distribution
# over [-1, 1]

B = 2 * np.random.rand(N,M) - 1

In [ ]:
# calculate dx/dt (equation 3, p. 18)

def calculate_response(x):
    return np.tanh(x)

def calculate_derivative(x, u, J, B, tau):
    r = calculate_response(x)
    return (1.0 / tau) * (-x + np.dot(J,r) + np.dot(B,u))

In [ ]:
# calculate euler approximation of the time step

def euler_timestep(x, u, J, B, tau, delta_t):
    return x + calculate_derivative(x, u, J, B, tau) * delta_t

In [ ]:
# run the simulation of the updates to the excitation

x_s = []  # keep track of x[0] over time for the plot
t_s = []  # trial numbers for the plot

for timestep in range(int(round(T/delta_t))):
    x = euler_timestep(x, u, J, B, tau, delta_t)
    x_s.append(x[0])
    t_s.append(timestep)
    #print "timestep:", timestep
    #print x

In [ ]:
# plot x_s against t_s
plt.plot(t_s, x_s)
plt.title('x[0] over time')
plt.xlabel('trial')
plt.ylabel('excitation level')
plt.show()

In [ ]:
def update_learning_potential(E, x_prev, x, x_average, N):
    s = lambda x: np.power(x, 3)  # supralinear modification
    R = np.repeat(calculate_response(x_prev), N).reshape(N,N).transpose()
    X_DIFF = np.repeat(x - x_average, N).reshape(N,N)
    return E + s(R * X_DIFF)

# runs a trial -- returns the weight change
def run_trial(x, u, J, B, tau, delta_t, N, reward_function, eta):
    E = np.zeros((N,N))  # how to initialize learning potential?
    x_alpha = 0.75       # decay of short-term running average of x
    for timestep in range(int(round(T/delta_t))):
        x_prev = np.copy(x)
        x = euler_timestep(x, u, J, B, tau, delta_t)
        x_average = x_alpha * x + (1 - x_alpha) * x_average
        E = update_learning_potential(E, x_prev, x, x_average, N)
        #TODO: where to apply neuron perturbation?
    r = reward_function()  # which args are needed?
    return eta * r * E
        